<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/hairy_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

import pandas as pd
import os
import shutil
if not os.path.exists('kaggle.json'):
  shutil.copy('/content/drive/My Drive/hairy_gan/kaggle.json', 'kaggle.json')
  # !pip install -q kaggle
  # files.upload()
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !kaggle datasets download -d jessicali9530/celeba-dataset --force
  !unzip celeba-dataset.zip
  !mv img_align_celeba celeba-dataset
  !mv list_eval_partition.csv celeba-dataset/list_eval_partition.csv
  !mv list_landmarks_align_celeba.csv celeba-dataset/list_landmarks_align_celeba.csv
  !mv list_attr_celeba.csv celeba-dataset/list_attr_celeba.csv
  !mv list_bbox_celeba.csv celeba-dataset/list_bbox_celeba.csv

  !mkdir celeba-dataset/train
  !mkdir celeba-dataset/validation
  !mkdir celeba-dataset/test

  partitions_df = pd.read_csv('celeba-dataset/list_eval_partition.csv') # 0 => train, 1 => validation, 2 => test
  for i, set_name in enumerate(['train', 'validation', 'test']):
    set_ids_df = partitions_df.loc[partitions_df['partition'] == i]['image_id']
    set_ids = set_ids_df.tolist()
    for id in set_ids:
      shutil.copy(os.path.join('celeba-dataset/img_align_celeba', id), os.path.join('celeba-dataset', f'{set_name}', id))

  !git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && pip install git+https://www.github.com/keras-team/keras-contrib.git \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install

  !pip install scipy==1.1.0

Streaming output truncated to the last 5000 lines.
  inflating: img_align_celeba/img_align_celeba/197910.jpg  
  inflating: img_align_celeba/img_align_celeba/197911.jpg  
  inflating: img_align_celeba/img_align_celeba/197912.jpg  
  inflating: img_align_celeba/img_align_celeba/197913.jpg  
  inflating: img_align_celeba/img_align_celeba/197914.jpg  
  inflating: img_align_celeba/img_align_celeba/197915.jpg  
  inflating: img_align_celeba/img_align_celeba/197916.jpg  
  inflating: img_align_celeba/img_align_celeba/197917.jpg  
  inflating: img_align_celeba/img_align_celeba/197918.jpg  
  inflating: img_align_celeba/img_align_celeba/197919.jpg  
  inflating: img_align_celeba/img_align_celeba/197920.jpg  
  inflating: img_align_celeba/img_align_celeba/197921.jpg  
  inflating: img_align_celeba/img_align_celeba/197922.jpg  
  inflating: img_align_celeba/img_align_celeba/197923.jpg  
  inflating: img_align_celeba/img_align_celeba/197924.jpg  
  inflating: img_align_celeba/img_align_celeba/19

In [2]:
from __future__ import print_function, division
import scipy
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Embedding
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam
from keras.models import load_model

import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from glob import glob
from PIL import Image

import tensorflow as tf
from tensorflow.python.keras.backend import set_session
# from tensorflow.python.keras.models import load_model
tf.compat.v1.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()

from tqdm import tqdm

Using TensorFlow backend.


In [73]:
class DataLoader():
    def __init__(self, dataset_name, img_res):
        self.dataset_name = dataset_name
        self.img_res = img_res
        self.complete_df = pd.read_csv('celeba-dataset/list_attr_celeba.csv')
        self.features = ['Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Bushy_Eyebrows', 'Eyeglasses', 'Gender', 'Mouth_Open', 'Mustache', 'No_Beard', 'Pale_Skin', 'Age']
        self.num_attrs = 9 # should equal to length of self.features

    def load_data(self, dataset_type, batch_size=1):
        data_type = dataset_type
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)
        total_samples = self.n_batches * batch_size

        path = np.random.choice(path, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs = []
            attribs = []
            for img_path in batch:
                img = self.imread(img_path)

                img = scipy.misc.imresize(img, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img = np.fliplr(img)

                imgs.append(img)

                # get attributes

                img_attribs = [(val + 1) // 2 for val in self.complete_df.loc[self.complete_df['image_id'] == os.path.basename(img_path)].filter(items=self.features).to_numpy()[0][1:].tolist()]

                attribs.append(np.array(img_attribs))

            imgs = np.array(imgs)/127.5 - 1.
            attribs = np.array(attribs)

            yield imgs, attribs

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [62]:
def build_encoder(img_shape, num_filters=64, kernel_size=4, strides=2):
  def build_conv(x, num_filters, kernel_size, strides):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    return x

  img = Input(shape=img_shape)
  x = build_conv(img, num_filters, kernel_size, strides)
  x = build_conv(x, num_filters * 2, kernel_size, strides)
  x = build_conv(x, num_filters * 4, kernel_size, strides)
  x = build_conv(x, num_filters * 8, kernel_size, strides)
  x = build_conv(x, num_filters * 16, kernel_size, strides)

  return Model(img, x)

def build_embedding(img, label, input_shape, attr_size):
  label_embedding = Embedding(2, np.prod(input_shape), input_length=attr_size)(label)
  label_embedding = Reshape(input_shape[:-1] + (attr_size * input_shape[-1], ))(label_embedding)
  emb_img = Concatenate(axis=-1)([img, label_embedding])
  return emb_img

def build_decoder(latent_space_shape, attr_size, num_filters=64, kernel_size=4, strides=1):
  def build_deconv(x, num_filters, kernel_size, strides):
    x = UpSampling2D(size=2)(x)
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    return x

  img = Input(shape=latent_space_shape)
  label = Input(shape=(attr_size, ), dtype='int32')

  emb_img = build_embedding(img, label, latent_space_shape, attr_size)

  x = build_deconv(emb_img, num_filters * 16, kernel_size=kernel_size, strides=strides)
  x = build_deconv(x, num_filters * 8, kernel_size=kernel_size, strides=strides)
  x = build_deconv(x, num_filters * 4, kernel_size=kernel_size, strides=strides)
  x = build_deconv(x, num_filters * 2, kernel_size=kernel_size, strides=strides)
  x = UpSampling2D(size=2)(x)
  x = Conv2D(3, kernel_size=kernel_size, strides=strides, padding='same', activation='tanh')(x)

  return Model([img, label], x)

def build_convnet(img, num_filters=64, kernel_size=4, strides=2):
  def build_conv(x, num_filters, kernel_size, strides):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = InstanceNormalization()(x)
    x = LeakyReLU()(x)
    return x
  
  x = build_conv(img, num_filters, kernel_size, strides)
  x = build_conv(x, num_filters * 2, kernel_size, strides)
  x = build_conv(x, num_filters * 4, kernel_size, strides)
  x = build_conv(x, num_filters * 8, kernel_size, strides)
  x = build_conv(x, num_filters * 16, kernel_size, strides)
  x = Flatten()(x)
  x = Dense(1024)(x)
  x = InstanceNormalization()(x)
  x = LeakyReLU()(x)

  return x

def build_dc(img_shape, attr_size, optimizer): # NOTE: we ignore inputting original image to discriminator head. why? cause im not sure if its important
  img = Input(shape=img_shape)
  label = Input(shape=(attr_size, ), dtype='int32')

  emb_img = build_embedding(img, label, img_shape, attr_size)
  x = build_convnet(emb_img)
  disc_output = Dense(1)(x)
  classif_output = Dense(attr_size, activation='sigmoid')(x)

  dc = Model([img, label], [disc_output, classif_output])

  dc.compile(loss=['binary_crossentropy', 'binary_crossentropy'], loss_weights=[1, 1], optimizer=optimizer)

  return dc

def build_combined_generator(img_shape, attr_size, genc, gdec, dc, optimizer):
  dc.trainable = False

  x_a = Input(shape=img_shape) # original image
  a = Input(shape=(attr_size, )) # original attr
  b = Input(shape=(attr_size, )) # requested attr
  
  z = genc(x_a) # latent space representation of original image
  x_b_hat = gdec([z, b]) # image with requested attr

  valid, b_hat = dc([x_b_hat, b]) # guess real or fake and if the requested features 

  x_a_hat = gdec([z, a]) # reconstr

  combined = Model(
      inputs=[x_a, a, b],
      outputs=[b_hat, valid, x_a_hat]
  )

  combined.compile(loss=['binary_crossentropy', 'binary_crossentropy', 'mae'], loss_weights=[10, 1, 100], optimizer=optimizer)

  return combined

In [53]:
def shuffle(elems):
  new_elems = elems.copy()
  np.random.shuffle(new_elems)
  return new_elems

def random_attrs(attr_size, count):
  return np.random.randint(0, 2, size=(count, attr_size))

In [70]:
def train_dc_step(batch_size, data_loader, genc, gdec, dc):
  imgs, attrs = next(batch_gen)
  new_attrs = random_attrs(attrs)

  real = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  x_a = imgs
  a = attrs
  b = new_attrs

  z = genc.predict(x_a)
  x_b_hat = gdec.predict([z, b])

  dc_real_history = dc.fit([x_a, a], [real, a])
  dc_fake_history = dc.fit([x_b_hat, b], [fake, b])


def train_encdec_step(batch_size, data_loader, combined):
  imgs, attrs = next(batch_gen)
  new_attrs = random_attrs(attrs)

  real = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  x_a = imgs
  a = attrs
  b = new_attrs

  g_real_loss = combined.fit([x_a, a, b], [b, real, x_a])


def train(num_epochs, num_batches, batch_size, batch_gen, genc, gdec, dc, combined):
  steps_per_epoch = num_batches

  for epoch in range(num_epochs):
    for step in tqdm(range(steps_per_epoch), desc=f'Train {epoch} / {num_epochs}', total=steps_per_epoch):
      train_dc_step(batch_size, batch_gen, genc, gdec, dc)
      train_encdec_step(batch_size, batch_gen, combined)

In [1]:
class HairyGan(): # based on AttGan
  def __init__(self):

    self.img_rows = 128
    self.img_cols = 128
    self.img_channels = 3

    self.img_shape = (self.img_rows, self.img_cols, self.img_channels)
    
    patch = int(self.img_rows / 2**4)
    self.disc_out = (patch, patch, 1) # output shape of discriminator

    self.data_loader = DataLoader(dataset_name='celeba-dataset', img_res=(self.img_rows, self.img_cols))

    self.optimizer = Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.999)

  def train(self, num_epochs, batch_size, sample_interval):
    # set up data loader
    dl = DataLoader('celeba-dataset', img_shape)
    gen = dl.load_batch()
    for i, elem in enumerate(gen):
      break

    # get data
    # imgs, attrs = next(gen)
    # new_attrs = random_attrs(attr_size, attrs.shape[0])

    enc = build_encoder(self.img_shape)
    
    # encodings = enc.predict(imgs)

    dec = build_decoder((4, 4, 1024), dl.num_attrs)

    # reconstrs = dec.predict([encodings, attrs])

    # new_imgs = dec.predict([encodings, new_attrs])

    dc = build_dc(img_shape, dl.num_attrs, Adam())

    combined = build_combined_generator(img_shape, dl.num_attrs, enc, dec, dc, self.optimizer)
    # combined.predict([imgs, attrs, new_attrs])    

    train(num_epochs=100, num_batches=dl.n_batches, batch_size=2, batch_gen=gen, genc=enc, gdec=dec, dc=dc, combined=combined)

  ## TODO: re-implement this method without domains
  def sample_images(self, epoch, batch_i):
    print(f'Epoch: {epoch} with batch: {batch_i}')
    rows, cols = 2, 3

    imgs_A = self.data_loader.load_data(batch_size=1, is_testing=True)
    imgs_B = self.data_loader.load_data(domain='B', batch_size=1, is_testing=True)

    fake_A = self.g_BA.predict(imgs_B)
    fake_B = self.g_AB.predict(imgs_A)

    reconstr_A = self.g_BA.predict(fake_B)
    reconstr_B = self.g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axes = plt.subplots(rows, cols)

    count = 0

    for i in range(rows):
      for j in range(cols):
        axes[i, j].imshow(gen_imgs[count])
        axes[i, j].set_title(titles[j])
        axes[i, j].axis('off')
        count += 1

    plt.show()
    

In [ ]:
project_path = '/content/drive/My Drive/hairy_gan'
gan = HairyGan()
gan.train(num_epochs=100, batch_size=64, sample_interval=100)

load


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [ ]:
gan.g_AB = load_model('/content/drive/My Drive/hairy_gan/g_AB.hdf5', custom_objects={'InstanceNormalization': InstanceNormalization})
gan.g_BA = load_model('/content/drive/My Drive/hairy_gan/g_BA.hdf5', custom_objects={'InstanceNormalization': InstanceNormalization})

In [ ]:
gan.sample_images()